In [1]:
# 以下代码实现读取图片与YOLOv5旋转格式的标签，并进行45度旋转增强，之后将增强后的图片保存至当前目录的out.jpg图片中。
# 相关函数在utils.plot与utils.datasets中实现
from utils.plot import GetLabels, VisualizeLabelsNP
import cv2 as cv
import sys
sys.path.append("/home/huasi/AllNeedCopy_datasets/dianli/YOLOv5_DOTA_OBB-master/")
from utils.datasets import rotate_augment

def get_basename(full_path):
    basename = os.path.basename(full_path)
    basename_no_ext = os.path.splitext(basename)[0]
    return basename_no_ext


# img_path = "/home/huasi/AllNeedCopy_datasets/dianli/YOLOv5_DOTA_OBB-master/DOTA_demo_view/dianli_datasets/train/images/img01709.jpg"
# label_path = "/home/huasi/AllNeedCopy_datasets/dianli/YOLOv5_DOTA_OBB-master/DOTA_demo_view/dianli_datasets/train/labels/img01709.txt"
# img_path = "./DOTA_demo_view/dianli_datasets/train/images/386402e2_0be6_4c87_8ccf_3b54ce22e081.jpg"
# label_path = "./DOTA_demo_view/dianli_datasets/newlabels/yolo_labels1/386402e2_0be6_4c87_8ccf_3b54ce22e081.txt" 
# output_path = "./DOTA_demo_view/dianli_datasets/newlabels/vis/"
# img = cv.imread(img_path)
# label_file = label_path
# labels = GetLabels(label_file)
# nimg, nlabels = rotate_augment(0, 1, img, labels)
# name=get_basename(img_path)
# VisualizeLabelsNP(nimg, nlabels, name, output_path)




In [2]:
# 以下代码段实现将某文件夹中的所有xml旋转目标标注文件转换为对应的YOLO格式文件；
import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join
import math
import numpy as np

def LongSideToPoly(xc_n, yc_n, w_n, h_n, W, H, theta1):
    xc = xc_n * W
    yc = yc_n * H
    w = w_n * W
    h = h_n * H
    # 1. calculate the angle theta2
    L = math.sqrt(w**2 + h**2)
    # print("L:    "+str(L))
    # print("w:    "+str(w))
    # print("h:    "+str(h))
    theta2 = math.degrees(math.atan(h/w))
    theta = theta1 + theta2
    # print(str(theta1))
    # print(str(theta))
    # 2. calculate (x1, y1)
    # print(str(L/2))
    dx1 = (L/2) * math.cos(math.radians(theta))
    dy1 = (L/2) * math.sin(math.radians(theta))
    # print("dx1an:"+str(math.cos(math.radians(theta))))
    # print("x1xc :"+str(dx1))
    x1 = xc + dx1
    y1 = yc + dy1
    # 3. calculate (x3, y3)
    x3 = xc - dx1
    y3 = yc - dy1
    # 4. calculate (x2, y2)
    dx2 = h * math.cos(math.radians(90 - theta1))
    dy2 = h * math.sin(math.radians(90 - theta1))
    x2 = x1 + dx2
    y2 = y1 - dy2
    # 5. calculate (x4, y4)
    dx4 = w * math.cos(math.radians(theta1))
    dy4 = w * math.sin(math.radians(theta1))
    x4 = x1 - dx4
    y4 = y1 - dy4

    return x1, y1, x2, y2, x3, y3, x4, y4

# calculate angle w.r.t. the long side
def LongSideTheta(poly_points):
    x1,y1,x2,y2,x3,y3,x4,y4 = poly_points
    # 1. calculate length of 1-2 and 2-3, find out which is the long side.
    l1_2 = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    l2_3 = math.sqrt((x3-x2)**2 + (y3-y2)**2)
    # calculate theta regarding the long side:
    if (l1_2 > l2_3):
        return math.degrees(math.atan((y1-y2)/(x2-x1)))
    else:
        return 180 - math.degrees(math.atan((y2-y3)/(x2-x3)))

# 读取XML标记文件并转换为YOLO格式；
# convert XML style rotated label to YOLO style label
# input:
#  size: the width and height of the input image
#  box: the upper-left and lower-right points of the bbox
# 注意：roLabelImage的XML标记格式中，角度并不一定匹配长边法！若不加以验证则可能导致训练失败！
def convert_robox(size, box):
    # compute the normalized factors
    dw = 1./(size[0])
    dh = 1./(size[1])
    # get the x,y,w,h in pixels
    cx = box[0]*dw
    cy = box[1]*dh
    w = box[2]*dw
    h = box[3]*dh
    ww = max(w, h)
    hh = min(w, h)
    a = math.degrees(box[4])
    # 将角度更新为一次长边法换算后的角度！
    poly_points = LongSideToPoly(cx, cy, ww, hh, size[0], size[1], a)
    a = LongSideTheta(poly_points)
    return (cx,cy,ww,hh,int(a))

# input:
#  size: the width and height of the input image
#  box: the upper-left and lower-right points of the bbox
def convert_bbox(size, box):
    # compute the normalized factors
    dw = 1./(size[0])
    dh = 1./(size[1])
    # compute the center of the bbox
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    # compute the width and height of the bbox
    w = box[1] - box[0]
    h = box[3] - box[2]
    # normalize the numbers
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    a = 0 if w>h else 90
    return (x,y,w,h,a)

# convert([1024, 1024], [383.8946, 354.4865, 622.7455, 32.5482,1.361357])

# parse XML file and extract
# Now we need to write a parsing funciton for the XML annotation files of the VOC format datasets.
# In particular, we need to extract the bboxes as well as the correponding classes of the bboxes, 
# and then convert them into YOLO format.
# Inputs:
classes = ['person','helmet','head','glove','wrongglove','clothes','wrongclothes','operatingbar','powerchecker','badge']
wrong_classes = {'clothes ': 'clothes', 'opeartingbar': 'operatingbar', 'operatingar': 'operatingbar', 'operatingbar ':'operatingbar', 'operatingbar  ':'operatingbar', 'operatingbar   ':'operatingbar', 'weongglove': 'wrongglove'}
#  dir_path: the path of directory containing all the xml annotation files
#  output_path: the path to save the generated YOLO txt annotation files
#  image_path: the path of an input image
def convert_xml_to_yolo(classes, dir_path, output_path, image_path):
    # e.g. if imagepath='./test.jpg', then basename_no_ext='test'
    basename = os.path.basename(image_path)
    basename_no_ext = os.path.splitext(basename)[0]

    # locate the corresponding xml annotation file of the image
    in_file = open(dir_path + '/' + basename_no_ext + '.xml')
    # set the output of the yolo annotation file
    out_file = open(output_path + basename_no_ext + '.txt', 'w')
    # start parsing the xml file and extract the bbox information
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    # enumerate all objects (bboxes) in the xml file
    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        # get class index
        cls = obj.find('name').text
        if cls not in classes:
            # if the cls is a wrong label, change it to the correct label!
            cls = wrong_classes[cls]
        cls_id = classes.index(cls)
        # get the corner coordinates and convert to yolo format
        xmlrbox = obj.find('robndbox')
        if (xmlrbox!=None):
            b = (float(xmlrbox.find('cx').text), float(xmlrbox.find('cy').text), float(xmlrbox.find('w').text), float(xmlrbox.find('h').text), float(xmlrbox.find('angle').text))
            bb = convert_robox((w,h), b)
            # finally write the "cls x y w h" as a single line into the yolo annotaiton file
            out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')
        xmlbox = obj.find('bndbox')
        if (xmlbox!=None):
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
            bb = convert_bbox((w,h), b)
            # finally write the "cls x y w h" as a single line into the yolo annotaiton file
            out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')


# image_path = "DOTA_demo_view/dianli_datasets/train/images/386402e2_0be6_4c87_8ccf_3b54ce22e081.jpg"
# dir_path = "DOTA_demo_view/dianli_datasets/newlabels/labels"
# output_path = "DOTA_demo_view/dianli_datasets/newlabels/yolo_labels1/"
# convert_xml_to_yolo(classes, dir_path, output_path, image_path)

# now we need to collect all the image paths into a list,
# and pass through the list while invoking convert_annotation function
#  dir_path: the directory to all the images
def getImagesInDir(dir_path):
    image_list = []
    for filename in glob.glob(dir_path + '/*.png'):
        image_list.append(filename)
    for filename in glob.glob(dir_path + '/*.jpg'):
        image_list.append(filename)
    return image_list

def getXMLsInDir(dir_path):
    xml_list = []
    for filename in glob.glob(dir_path + '/*.xml'):
        xml_list.append(filename)
    return xml_list
# image_paths = getImagesInDir("./DOTA_demo_view/dianli_datasets/train/images/")
# print(image_paths)

dir_path = "DOTA_demo_view/dianli_datasets/operatingbars/newsamples_labels/"
xml_path = "DOTA_demo_view/dianli_datasets/operatingbars/newsamples_labels/"
output_path = "DOTA_demo_view/dianli_datasets/operatingbars/newsamples_yolo/"
# if the output directory does not exists, we should create it
if not os.path.exists(output_path):
    os.makedirs(output_path)

xml_paths = getXMLsInDir(xml_path)
for xml_path in xml_paths:
    convert_xml_to_yolo(classes, dir_path, output_path, xml_path)


In [6]:
# 以下实现将转换完毕的标注文件可视化并在tensorboard中显示

import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join
import math
import cv2 as cv

def get_basename(full_path):
    basename = os.path.basename(full_path)
    basename_no_ext = os.path.splitext(basename)[0]
    return basename_no_ext

# 首先生成所有需要展示的yolo标注列表
def getTXTsInDir(dir_path):
    txt_list = []
    for filename in glob.glob(dir_path + '/*.txt'):
        txt_list.append(filename)
    return txt_list

txt_path = "DOTA_demo_view/dianli_datasets/newlabels/yolo_labels16/"
txt_paths = getTXTsInDir(txt_path)

# 遍历所有txt列表中的所有标注文件，并生成相应的图片保存到指定位置
img_output_path = "./DOTA_demo_view/dianli_datasets/newlabels/vis/"
img_input_dir = "./DOTA_demo_view/dianli_datasets/train/images/"
for txt_path in txt_paths:
    name = get_basename(txt_path)
    img_path = img_input_dir + name + ".jpg"
    # print(img_path)
    img = cv.imread(img_path)
    labels = GetLabels(txt_path)
    VisualizeLabelsNP(img, labels, name, img_output_path)

In [7]:
# 将生成的可视化标注图片以4x4的格式展示到tensorboard中
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("tensorboard/baiyu")

import torch
import torchvision
from torchvision.io import read_image
from torchvision.utils import make_grid

img_dir = "./DOTA_demo_view/dianli_datasets/newlabels/vis/"
img_paths = getImagesInDir(img_dir)
imgs = []
count = 0
batch_id = 0
for img_path in img_paths:
    if count==16:
        grid = make_grid(imgs)
        writer.add_image('helmet_images', grid, batch_id)
        print(f"generating {batch_id}th image grid...")
        count = 0
        batch_id += 1
        imgs = []
    img = read_image(img_path)
    imgs.append(img)
    count +=1


generating 0th image grid...
generating 1th image grid...
generating 2th image grid...
generating 3th image grid...
generating 4th image grid...
generating 5th image grid...


In [8]:
!python train2.py --weights=./weights/yolov5s.pt --multi-scale

^C
Traceback (most recent call last):
  File "train2.py", line 23, in <module>
    from models.yolo import Model
  File "/home/huasi/AllNeedCopy_datasets/dianli/YOLOv5_DOTA_OBB-master/models/yolo.py", line 14, in <module>
    from models.common import *
  File "/home/huasi/AllNeedCopy_datasets/dianli/YOLOv5_DOTA_OBB-master/models/common.py", line 6, in <module>
    from utils.general import non_max_suppression
  File "/home/huasi/AllNeedCopy_datasets/dianli/YOLOv5_DOTA_OBB-master/utils/general.py", line 15, in <module>
    import matplotlib
  File "/home/huasi/miniconda3/envs/v5_rotate/lib/python3.8/site-packages/matplotlib/__init__.py", line 109, in <module>
    from . import _api, _version, cbook, docstring, rcsetup
  File "/home/huasi/miniconda3/envs/v5_rotate/lib/python3.8/site-packages/matplotlib/rcsetup.py", line 28, in <module>
    from matplotlib.fontconfig_pattern import parse_fontconfig_pattern
  File "/home/huasi/miniconda3/envs/v5_rotate/lib/python3.8/site-packages/matplotl

In [1]:
# 检测推理模型的效果
!python detect.py --weight="./runs/exp37/weights/best.pt" \
     --source="./DOTA_demo_view/dianli_datasets/raw_imgs/dj/newimgs/" \
     --output="./DOTA_demo_view/dianli_datasets/det_visualization/"

python detect.py --weight="/home/huasi/users/lizihan/yolov5/runs/train/exp41/weights/best.pt" --source="/home/huasi/AllNeedCopy_datasets/dianli/YOLOv5_DOTA_OBB-master/DOTA_demo_view/dianli_datasets/raw_imgs/dj/samples/" --project="/home/huasi/AllNeedCopy_datasets/dianli/YOLOv5_DOTA_OBB-master/DOTA_demo_view/dianli_datasets/det_visualization/"

SyntaxError: invalid syntax (3535809411.py, line 4)

In [2]:
!python detect.py --weights "./runs/exp46/weights/best.pt" --source "/home/huasi/AllNeedCopy_datasets/dianli/YOLOv5_DOTA_OBB-master/DOTA_demo_view/dianli_datasets/raw_imgs/dj/samples/" --output "/home/huasi/AllNeedCopy_datasets/dianli/YOLOv5_DOTA_OBB-master/DOTA_demo_view/dianli_datasets/det_visualization/"

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.2, device='1,2,3,4,5,6,7', img_size=640, iou_thres=0.1, output='/home/huasi/AllNeedCopy_datasets/dianli/YOLOv5_DOTA_OBB-master/DOTA_demo_view/dianli_datasets/det_visualization/', save_txt=False, source='/home/huasi/AllNeedCopy_datasets/dianli/YOLOv5_DOTA_OBB-master/DOTA_demo_view/dianli_datasets/raw_imgs/dj/samples/', update=False, view_img=False, weights=['./runs/exp41/weights/best.pt'])
Using CUDA device0 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3090', total_memory=24268MB)
           device1 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3090', total_memory=24268MB)
           device2 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3090', total_memory=24268MB)
           device3 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3090', total_memory=24268MB)
           device4 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3090', total_memory=24268MB)
           device5 _CudaDeviceProperties(name='NVIDIA GeForce

In [13]:
!python train2.py --device 1,2,3,4 --batch-size 60

Using CUDA device0 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3090', total_memory=24268MB)
           device1 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3090', total_memory=24268MB)
           device2 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3090', total_memory=24268MB)
           device3 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3090', total_memory=24268MB)

Namespace(adam=False, batch_size=40, bucket='', cache_images=False, cfg='./models/yolov5m.yaml', data='data/secure_check.yaml', device='1,2,3,4', epochs=10, evolve=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], local_rank=-1, logdir='runs/', multi_scale=False, name='', noautoanchor=False, nosave=False, notest=True, rect=False, resume=False, single_cls=False, sync_bn=False, total_batch_size=40, weights='./weights/yolov5m.pt', workers=8, world_size=1)
Start Tensorboard with "tensorboard --logdir runs/", view at http://localhost:6006/
2023-03-22 22:04:00.378260: I tens

In [29]:
!python /home/huasi/AllNeedCopy_datasets/dianli/YOLOv5_DOTA_OBB-master/yolov5/yolov5_obb-master/train.py --img 640 --batch 64 --epochs 300  

train: weights=yolov5/yolov5_obb-master/weights/yolov5n.pt, cfg=, data=yolov5/yolov5_obb-master/data/DroneVehicle_poly.yaml, hyp=yolov5/yolov5_obb-master/data/hyps/obb/hyp.finetune_dota.yaml, epochs=100, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=1,2,3,4, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=yolov5/yolov5_obb-master/runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=300, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 2023-3-23 torch 1.10.1+cu113 CUDA:1 (NVIDIA GeForce RTX 3090, 24268MiB)
                                       CUDA:2 (NVIDIA GeForce RTX 3090, 24268MiB)
                                  